In [3]:
#nn.BatchNorm1d(120)
#nn.Dropout(p=0.5)
#max_pool2d(out, 2)
#.view(out.size(0), -1)
#nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
#nn.Tanh(),

from __future__ import print_function
import torch  # import the torch library
import torch.nn as nn # use the nn module (class)
import torch.nn.functional as F    # use the nn module as function
import torch.optim as optim # optimization (i.e., SGD, ada,)
import torchvision # load the dataset
import torchvision.transforms as transforms # adjust the input image
import time # check the processing overhead

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# total number of classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define the class for LeNet
class LeNet(nn.Module):
    def __init__(self):# will be called when you create an object
        super(LeNet, self).__init__()

        self.c1 = nn.Conv2d(3,8,11,1,5)
        self.c2 = nn.Conv2d(8,16,7,1,3)
        self.c3 = nn.Conv2d(16,16,5,1,2)
        self.c4 = nn.Conv2d(16,16,5,1,0)
        self.c5 = nn.Conv2d(16,16,5,1,0)
        self.c6 = nn.Conv2d(16,160,5,1,0)
        

        self.F1 = nn.Linear(160, 160)
        self.F2 = nn.Linear(160,10)
        self.MyRelu=nn.ReLU()


    def forward(self, x):  

        x=self.MyRelu(self.c1(x)) # 1st Conv Layer
        x=self.MyRelu(self.c2(x)) # 2nd Conv Layer
        x=self.MyRelu(self.c3(x)) # 3rd Conv Layer
        x=self.MyRelu(self.c4(x)) # 4th Conv Layer
        
        x=F.max_pool2d(x,2,2)     # Max Pool 1
        
        x=self.MyRelu(self.c5(x)) # 5th Conv Layer
        
        x=F.max_pool2d(x,2,2)     # Max Pool 2
        
        x=self.MyRelu(self.c6(x)) # 6th Conv Layer

        x=torch.flatten(x, 1)     # Flatten
        
        x=self.MyRelu(self.F1(x)) # Fully Connected Layer 1 
        x=self.F2(x)              # Fully Connected Layer 2

        return x



def train(model, device, train_loader, optimizer, epoch):
    model.train() # set the model into training model (evaluation model in the testing)
    count = 0

    loss_Fn=nn.CrossEntropyLoss()
    #nn.functional.cross_entropy()
    for batch_idx, (data, target) in enumerate(train_loader):
      #data is the image
      #target is the ground truth
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        Predict=model(data)
        loss=loss_Fn(Predict,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.01
    save_model = False
    use_cuda = torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device('cuda' if use_cuda else 'cpu')

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time()
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))
if __name__ == '__main__':
    main()

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.304930
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.307133
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.300615
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.308159
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.302222
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.303153
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.301867
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.299970
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.300152
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.307656
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.300843
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.302276
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2.300704
Train Epoch: 1 [16640/50000 (33%)]	Loss: 2.302680
Train Epoch: 1 [17920/50000 (36%)]	Loss: 2.304365
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.304463
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.301723
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.303496
Train Epoch: 1 [23


Test set: Average loss: -2.1816, Accuracy: 4812/10000 (48%)

Train Epoch: 5 [0/50000 (0%)]	Loss: 1.449472
Train Epoch: 5 [1280/50000 (3%)]	Loss: 1.394393
Train Epoch: 5 [2560/50000 (5%)]	Loss: 1.519607
Train Epoch: 5 [3840/50000 (8%)]	Loss: 1.409691
Train Epoch: 5 [5120/50000 (10%)]	Loss: 1.399218
Train Epoch: 5 [6400/50000 (13%)]	Loss: 1.435061
Train Epoch: 5 [7680/50000 (15%)]	Loss: 1.496282
Train Epoch: 5 [8960/50000 (18%)]	Loss: 1.463622
Train Epoch: 5 [10240/50000 (20%)]	Loss: 1.556027
Train Epoch: 5 [11520/50000 (23%)]	Loss: 1.505182
Train Epoch: 5 [12800/50000 (26%)]	Loss: 1.459366
Train Epoch: 5 [14080/50000 (28%)]	Loss: 1.425509
Train Epoch: 5 [15360/50000 (31%)]	Loss: 1.478308
Train Epoch: 5 [16640/50000 (33%)]	Loss: 1.352839
Train Epoch: 5 [17920/50000 (36%)]	Loss: 1.372948
Train Epoch: 5 [19200/50000 (38%)]	Loss: 1.330836
Train Epoch: 5 [20480/50000 (41%)]	Loss: 1.439931
Train Epoch: 5 [21760/50000 (43%)]	Loss: 1.510360
Train Epoch: 5 [23040/50000 (46%)]	Loss: 1.529785
Tra

Train Epoch: 9 [1280/50000 (3%)]	Loss: 1.341895
Train Epoch: 9 [2560/50000 (5%)]	Loss: 1.342332
Train Epoch: 9 [3840/50000 (8%)]	Loss: 1.058003
Train Epoch: 9 [5120/50000 (10%)]	Loss: 1.299018
Train Epoch: 9 [6400/50000 (13%)]	Loss: 1.250288
Train Epoch: 9 [7680/50000 (15%)]	Loss: 1.150312
Train Epoch: 9 [8960/50000 (18%)]	Loss: 1.286855
Train Epoch: 9 [10240/50000 (20%)]	Loss: 0.937041
Train Epoch: 9 [11520/50000 (23%)]	Loss: 1.121266
Train Epoch: 9 [12800/50000 (26%)]	Loss: 1.157283
Train Epoch: 9 [14080/50000 (28%)]	Loss: 1.325845
Train Epoch: 9 [15360/50000 (31%)]	Loss: 1.348495
Train Epoch: 9 [16640/50000 (33%)]	Loss: 1.251776
Train Epoch: 9 [17920/50000 (36%)]	Loss: 1.269993
Train Epoch: 9 [19200/50000 (38%)]	Loss: 1.161815
Train Epoch: 9 [20480/50000 (41%)]	Loss: 1.226827
Train Epoch: 9 [21760/50000 (43%)]	Loss: 1.184088
Train Epoch: 9 [23040/50000 (46%)]	Loss: 1.350905
Train Epoch: 9 [24320/50000 (49%)]	Loss: 1.132196
Train Epoch: 9 [25600/50000 (51%)]	Loss: 1.332675
Train Epoc


Test set: Average loss: -4.1647, Accuracy: 6529/10000 (65%)

Train Epoch: 13 [0/50000 (0%)]	Loss: 1.038345
Train Epoch: 13 [1280/50000 (3%)]	Loss: 1.058928
Train Epoch: 13 [2560/50000 (5%)]	Loss: 0.900032
Train Epoch: 13 [3840/50000 (8%)]	Loss: 0.849518
Train Epoch: 13 [5120/50000 (10%)]	Loss: 0.968215
Train Epoch: 13 [6400/50000 (13%)]	Loss: 0.949115
Train Epoch: 13 [7680/50000 (15%)]	Loss: 0.993985
Train Epoch: 13 [8960/50000 (18%)]	Loss: 1.148851
Train Epoch: 13 [10240/50000 (20%)]	Loss: 0.983057
Train Epoch: 13 [11520/50000 (23%)]	Loss: 1.011580
Train Epoch: 13 [12800/50000 (26%)]	Loss: 1.103337
Train Epoch: 13 [14080/50000 (28%)]	Loss: 1.109682
Train Epoch: 13 [15360/50000 (31%)]	Loss: 1.042653
Train Epoch: 13 [16640/50000 (33%)]	Loss: 0.910358
Train Epoch: 13 [17920/50000 (36%)]	Loss: 1.017853
Train Epoch: 13 [19200/50000 (38%)]	Loss: 1.121873
Train Epoch: 13 [20480/50000 (41%)]	Loss: 1.166647
Train Epoch: 13 [21760/50000 (43%)]	Loss: 1.015463
Train Epoch: 13 [23040/50000 (46%)]

Train Epoch: 16 [47360/50000 (95%)]	Loss: 0.929519
Train Epoch: 16 [48640/50000 (97%)]	Loss: 1.000535
Train Epoch: 16 [31200/50000 (100%)]	Loss: 0.989182

Test set: Average loss: -4.8419, Accuracy: 6904/10000 (69%)

Train Epoch: 17 [0/50000 (0%)]	Loss: 0.860918
Train Epoch: 17 [1280/50000 (3%)]	Loss: 0.874411
Train Epoch: 17 [2560/50000 (5%)]	Loss: 0.882706
Train Epoch: 17 [3840/50000 (8%)]	Loss: 1.015805
Train Epoch: 17 [5120/50000 (10%)]	Loss: 1.140112
Train Epoch: 17 [6400/50000 (13%)]	Loss: 0.964531
Train Epoch: 17 [7680/50000 (15%)]	Loss: 0.836081
Train Epoch: 17 [8960/50000 (18%)]	Loss: 0.992696
Train Epoch: 17 [10240/50000 (20%)]	Loss: 0.932073
Train Epoch: 17 [11520/50000 (23%)]	Loss: 0.962885
Train Epoch: 17 [12800/50000 (26%)]	Loss: 0.814670
Train Epoch: 17 [14080/50000 (28%)]	Loss: 0.910387
Train Epoch: 17 [15360/50000 (31%)]	Loss: 0.943765
Train Epoch: 17 [16640/50000 (33%)]	Loss: 0.872282
Train Epoch: 17 [17920/50000 (36%)]	Loss: 0.903446
Train Epoch: 17 [19200/50000 (38%)

...

Train Epoch: 32 [32000/50000 (64%)]	Loss: 1.066222
Train Epoch: 32 [33280/50000 (66%)]	Loss: 0.842733
Train Epoch: 32 [34560/50000 (69%)]	Loss: 0.832244
Train Epoch: 32 [35840/50000 (72%)]	Loss: 0.908539
Train Epoch: 32 [37120/50000 (74%)]	Loss: 0.937101
Train Epoch: 32 [38400/50000 (77%)]	Loss: 0.756581
Train Epoch: 32 [39680/50000 (79%)]	Loss: 0.781187
Train Epoch: 32 [40960/50000 (82%)]	Loss: 0.898761
Train Epoch: 32 [42240/50000 (84%)]	Loss: 0.703629
Train Epoch: 32 [43520/50000 (87%)]	Loss: 0.839059
Train Epoch: 32 [44800/50000 (90%)]	Loss: 0.749345
Train Epoch: 32 [46080/50000 (92%)]	Loss: 0.809670
Train Epoch: 32 [47360/50000 (95%)]	Loss: 0.927096
Train Epoch: 32 [48640/50000 (97%)]	Loss: 0.912628
Train Epoch: 32 [31200/50000 (100%)]	Loss: 0.934743

Test set: Average loss: -6.0593, Accuracy: 7237/10000 (72%)

Train Epoch: 33 [0/50000 (0%)]	Loss: 0.651892
Train Epoch: 33 [1280/50000 (3%)]	Loss: 0.762113
Train Epoch: 33 [2560/50000 (5%)]	Loss: 0.739175
Train Epoch: 33 [3840/50000 

Train Epoch: 36 [28160/50000 (56%)]	Loss: 0.814965
Train Epoch: 36 [29440/50000 (59%)]	Loss: 0.776754
Train Epoch: 36 [30720/50000 (61%)]	Loss: 0.843693
Train Epoch: 36 [32000/50000 (64%)]	Loss: 0.593056
Train Epoch: 36 [33280/50000 (66%)]	Loss: 0.686213
Train Epoch: 36 [34560/50000 (69%)]	Loss: 0.688571
Train Epoch: 36 [35840/50000 (72%)]	Loss: 0.665271
Train Epoch: 36 [37120/50000 (74%)]	Loss: 0.655606
Train Epoch: 36 [38400/50000 (77%)]	Loss: 0.822447
Train Epoch: 36 [39680/50000 (79%)]	Loss: 0.793859
Train Epoch: 36 [40960/50000 (82%)]	Loss: 0.881629
Train Epoch: 36 [42240/50000 (84%)]	Loss: 0.901326
Train Epoch: 36 [43520/50000 (87%)]	Loss: 0.805094
Train Epoch: 36 [44800/50000 (90%)]	Loss: 0.800512
Train Epoch: 36 [46080/50000 (92%)]	Loss: 0.736339
Train Epoch: 36 [47360/50000 (95%)]	Loss: 0.616617
Train Epoch: 36 [48640/50000 (97%)]	Loss: 0.776525
Train Epoch: 36 [31200/50000 (100%)]	Loss: 0.785733

Test set: Average loss: -6.1538, Accuracy: 7508/10000 (75%)

Train Epoch: 37 [0/

Train Epoch: 40 [24320/50000 (49%)]	Loss: 0.554804
Train Epoch: 40 [25600/50000 (51%)]	Loss: 0.854081
Train Epoch: 40 [26880/50000 (54%)]	Loss: 0.735690
Train Epoch: 40 [28160/50000 (56%)]	Loss: 0.819700
Train Epoch: 40 [29440/50000 (59%)]	Loss: 0.765385
Train Epoch: 40 [30720/50000 (61%)]	Loss: 0.896807
Train Epoch: 40 [32000/50000 (64%)]	Loss: 0.760077
Train Epoch: 40 [33280/50000 (66%)]	Loss: 0.639212
Train Epoch: 40 [34560/50000 (69%)]	Loss: 0.721896
Train Epoch: 40 [35840/50000 (72%)]	Loss: 0.848000
Train Epoch: 40 [37120/50000 (74%)]	Loss: 0.609314
Train Epoch: 40 [38400/50000 (77%)]	Loss: 0.634959
Train Epoch: 40 [39680/50000 (79%)]	Loss: 0.704800
Train Epoch: 40 [40960/50000 (82%)]	Loss: 0.728856
Train Epoch: 40 [42240/50000 (84%)]	Loss: 0.769867
Train Epoch: 40 [43520/50000 (87%)]	Loss: 0.725605
Train Epoch: 40 [44800/50000 (90%)]	Loss: 0.866206
Train Epoch: 40 [46080/50000 (92%)]	Loss: 0.760712
Train Epoch: 40 [47360/50000 (95%)]	Loss: 0.700480
Train Epoch: 40 [48640/50000 (9

Train Epoch: 44 [21760/50000 (43%)]	Loss: 0.760202
Train Epoch: 44 [23040/50000 (46%)]	Loss: 0.788233
Train Epoch: 44 [24320/50000 (49%)]	Loss: 0.745200
Train Epoch: 44 [25600/50000 (51%)]	Loss: 0.755870
Train Epoch: 44 [26880/50000 (54%)]	Loss: 0.853992
Train Epoch: 44 [28160/50000 (56%)]	Loss: 0.719829
Train Epoch: 44 [29440/50000 (59%)]	Loss: 0.848743
Train Epoch: 44 [30720/50000 (61%)]	Loss: 0.712678
Train Epoch: 44 [32000/50000 (64%)]	Loss: 0.811725
Train Epoch: 44 [33280/50000 (66%)]	Loss: 0.852487
Train Epoch: 44 [34560/50000 (69%)]	Loss: 0.716696
Train Epoch: 44 [35840/50000 (72%)]	Loss: 0.719380
Train Epoch: 44 [37120/50000 (74%)]	Loss: 0.872897
Train Epoch: 44 [38400/50000 (77%)]	Loss: 0.676785
Train Epoch: 44 [39680/50000 (79%)]	Loss: 0.755485
Train Epoch: 44 [40960/50000 (82%)]	Loss: 0.665882
Train Epoch: 44 [42240/50000 (84%)]	Loss: 0.816640
Train Epoch: 44 [43520/50000 (87%)]	Loss: 0.927420
Train Epoch: 44 [44800/50000 (90%)]	Loss: 0.932091
Train Epoch: 44 [46080/50000 (9

Train Epoch: 48 [17920/50000 (36%)]	Loss: 0.672552
Train Epoch: 48 [19200/50000 (38%)]	Loss: 0.723815
Train Epoch: 48 [20480/50000 (41%)]	Loss: 0.757631
Train Epoch: 48 [21760/50000 (43%)]	Loss: 0.794135
Train Epoch: 48 [23040/50000 (46%)]	Loss: 0.860009
Train Epoch: 48 [24320/50000 (49%)]	Loss: 0.799863
Train Epoch: 48 [25600/50000 (51%)]	Loss: 0.725596
Train Epoch: 48 [26880/50000 (54%)]	Loss: 0.618312
Train Epoch: 48 [28160/50000 (56%)]	Loss: 0.767898
Train Epoch: 48 [29440/50000 (59%)]	Loss: 0.654487
Train Epoch: 48 [30720/50000 (61%)]	Loss: 0.761601
Train Epoch: 48 [32000/50000 (64%)]	Loss: 0.823549
Train Epoch: 48 [33280/50000 (66%)]	Loss: 0.771200
Train Epoch: 48 [34560/50000 (69%)]	Loss: 0.878569
Train Epoch: 48 [35840/50000 (72%)]	Loss: 0.822919
Train Epoch: 48 [37120/50000 (74%)]	Loss: 0.593560
Train Epoch: 48 [38400/50000 (77%)]	Loss: 0.705258
Train Epoch: 48 [39680/50000 (79%)]	Loss: 0.721117
Train Epoch: 48 [40960/50000 (82%)]	Loss: 0.556186
Train Epoch: 48 [42240/50000 (8

**Conclusion** Obtained a 75% accuracy with the custom conv network !